Name: **Muhammad Junaid Aftab**
UID:  **117396188**

# Homework 8:  ADMM

In [1]:
from utility import *
import numpy as np
from numpy import sqrt, sum, abs, sign, max, maximum, minimum, logspace, exp, log, log10, zeros
from numpy.linalg import norm
from numpy.random import randn, rand, normal, randint
import urllib
import matplotlib.pyplot as plt
np.random.seed(0)
def good_job(path):
    f = urllib.request.urlopen(path)
    a = plt.imread(io.BytesIO(f.read()))
    fig = plt.imshow(a)
    fig.axes.get_xaxis().set_visible(False)
    fig.axes.get_yaxis().set_visible(False)
    plt.show()
np.random.seed(0)

In this problem,
you'll write an "unwrapped ADMM" solver for the support-vector machine problem
$$
 \min \quad \frac{1}{2} \|x\|^2+Ch(Ax)
$$ 
where $h(z) = \sum_i \max\{1-z_i,0\}$ is the hinge loss function, and  $A = YX$ is the product of the (diagonal) label matrix with the data matrix.  Your solver will be based on the constrained formulation

\begin{align}
 \min &\quad \frac{1}{2} \|x\|^2+Ch(y)\\
 \text{subject to}&\quad   y-Ax=0.
\end{align}

Start by running the block below to produce a test problem.

In [2]:
# Define a classification problem
X, y = create_classification_problem(100, 10, cond_number=10)
A = y*X
t = 1/norm(A.T@A)
C = 1

### Write the scaled augmented Lagrangian for the constrained problem
Use $\lambda$ to denote the Lagrange multiplier.
 
The augmented Lagranigian is,

$$
\mathcal{L}(x, y, \lambda) 
=
\frac{1}{2} \|x\|^2+Ch(y)
+
\frac{\tau}{2} \| y - Ax + \lambda/\tau \|^2.
$$

As in the slides, we replace $\lambda$ by $\hat{\lambda} = \lambda/\tau$ below.

### Write the system of equations that needs to be solved to update $x$

Recall that the ADMM solver for the problem is given by,

$$
x^{k+1} = \text{arg min}_x \dfrac{1}{2}||x||^2 + \dfrac{\tau}{2}||y^k - Ax + \lambda^k||^2, \\
y^{k+1} = \text{arg min}_y Ch(y) + \dfrac{\tau}{2}||y - Ax^{k+1} + \lambda^k||^2, \\
\lambda^{k+1} = \hat{\lambda}^k  + y^{k+1} - Ax^{k+1}.
$$

To update $x$, we must solve,

$$
\text{argmin}_{x}
\frac{1}{2} \|x\|^2 +
\frac{\tau}{2}
\| y - Ax + \hat{\lambda} \|^2.
$$

Taking derivaties, the optimality condition is,

$$
x - \tau A^T( y - Ax + \hat{\lambda} ) = 0.
$$

Rearranging, we get,

$$
(I + \tau A^T A )x = \tau A^T( \hat{\lambda} + y ).
$$

This implies

$$
x = \tau (I + \tau A^T A )^{-1} A^T( \hat{\lambda} + y ).
$$

### Write a routine to evaluate the prox operator of the hinge loss. Your routine will return the solution to the prox problem
$$\text{prox}_h(z,t) = \arg\min_x h(x) + \frac{1}{2t}\|x-z\|^2.$$
You cannot use an iterative method to compute this value.  Note: this can be done with one line of code.  The prox operator for the hinge loss is a lot like the prox operator for L1.

In [3]:
def hprox(z,t):
    return np.fmax( np.fmin( np.fmin(z,1-t) + t, 1), z )

### Now run these unit tests

In [4]:
assert hprox(0,1)==1, "Your prox operator failed unit test 1"
assert hprox(1,1)==1, "Your prox operator failed unit test 2"
assert hprox(-1,1)==0, "Your prox operator failed unit test 3"
assert hprox(1,2)==1, "Your prox operator failed unit test 4"
assert hprox(1,3)==1, "Your prox operator failed unit test 5"
assert hprox(4,3)==4, "Your prox operator failed unit test 6"

### Write an ADMM loop to solve the SVM problem above.  
Use your prox operator for the $y$ update.
Run the solver until the primal and dual residuals satisfy
$$p^k \le 10^{-5} \max_{i<k}p^i$$
$$d^k \le 10^{-5} \max_{i<k}d^i.$$
The residuals are given by 
$$p^k = \|Ax^k-y^k\|$$
$$d^k = \|\tau A^T(y^k-y^{k-1})\|.$$

In [11]:
def ADMM(hprox, x, y, l, t, max_iters = 10000, tol=1e-6):
    
    l = l/t
    
    I = np.identity(np.shape(A.T@A)[0])
    M = np.linalg.inv(I + t*A.T@A)
    
    p = []
    d = []
    
    k = 0
    Boolean = True
    
    while k <= max_iters and Boolean == True:
        
        x = t*(M@A.T@(l + y))
        ynew = hprox(A@x - l,1/t)
        l = l - A@x + ynew
        
        p.append(np.linalg.norm(A@x - ynew))
        d.append(np.linalg.norm(t*(A.T@(ynew - y))))

        if p[-1] <= (10*(-5))*np.max(p) and d[-1] <= (10*(-5))*np.max(d):
            
            Boolean = False
            
        y = ynew
        k = k + 1
        
    
    return x, y, l

### Now run this unit test

In [12]:
# There are two different optimality conditions that could be satisfied depending on how you formulate the lagrangian

x0 = zeros((10,1))
y0 = zeros((100,1))
l0 = zeros((100,1))

x, y, l = ADMM(hprox, x0, y0, l0, t, max_iters = 10000, tol=1e-6)

error1 = norm(x+t*A.T@(A@x-y-l))
error2 = norm(x+t*A.T@(A@x-y+l))
print(error1)
assert min(error1/norm(x), error2/norm(x))<1e-3, 'Your ADMM solver did not produce an accurate solution.'
z1 = A@x-l
z2 = A@x+l
error1 = norm(y-z1 - maximum(minimum(1-z1,C/t),0))
error2 = norm(y-z2 - maximum(minimum(1-z2,C/t),0))
assert min(error1/norm(y),error2/norm(y)) <1e-3, 'Your ADMM solver did not produce an accurate solution.'
print('Congrats! Your solver works!!')
#good_job("https://www.cs.umd.edu/~tomg/img/important_memes/ron_awesome.png")

9.364490596432115e-16
Congrats! Your solver works!!
